<a href="https://colab.research.google.com/github/hahuhi518/COMP5703-Capstone/blob/main/Point_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference paper:https://arxiv.org/abs/2012.11967

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import libraries

In [ ]:
!pip install biobert-pytorch pytorch-transformers
!pip install transformers
!pip install tweet-preprocessor
!pip install emoji --upgrade
!pip install ekphrasis
from transformers import BertModel, BertTokenizer
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


import numpy as np
import pandas as pd

import preprocessor as pp
import re
import regex
import emoji
import unicodedata
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from ekphrasis.classes.segmenter import Segmenter


import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

import os
import sys
import pickle
import copy
import time
import datetime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ========== Setting up parameter ========== #
MY_PATH = '/content/drive/MyDrive/Datasets/'

## Please remember to change class name when doing binary
# CLASS_NAME = ['Figurative', 'Non-health', 'Health']
# CLASS_NAME = ['Non-health', 'Health'] # Binary label 01 & label 2
CLASS_NAME = ['Figurative', 'Health'] # Binary label 0 & label 2
NUM_CLASSES = len(CLASS_NAME)

# link for the CT-BERT https://huggingface.co/digitalepidemiologylab/covid-twitter-bert-v2
BERT_MODEL_NAME = 'digitalepidemiologylab/covid-twitter-bert-v2'
MAX_LENGTH = 128
BATCH_SIZE = 8
EPOCH = 10

# False to disable Self-training (TGS)
enable_TGS = True
# False to disable Contrastive Objective (CO)
enable_CO = True
n_splits = 5 #KFold
Alpha = 0.2
beta_loss = 0.1
freeze_layer_count = 0

SEED=17

MODEL_ID = 'CT_BERT_D3_TGS'
MODEL_ID = MODEL_ID +str(int(enable_TGS))+'_changed_CO'+str(int(enable_CO))+'_Alpha'+str(Alpha)+'_Beta'+str(beta_loss)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
!nvidia-smi

# Import data

In [ ]:
data_org_d1 = pd.read_csv(MY_PATH+ 'Health Mention Twitter Dataset - original (D1).csv', header=0)
data_org_d2 = pd.read_csv(MY_PATH+ 'Health Mention Reddit Dataset - original (D2).csv', header=0)

In [ ]:
# HMC2019 dataset
data_org_d1 = pd.read_csv(MY_PATH+ 'Health Mention Twitter Dataset - original (D1).csv', header=0)
data_org_d2 = pd.read_csv(MY_PATH+ 'Health Mention Reddit Dataset - original (D2).csv', header=0)

#tweets_org = df3['tweet'].tolist()
#df_gb_org = df3.groupby(['disease','label']).size().unstack(level=1)

# Combination of both original and extended dataset

df3 = pd.concat([data_org_d1,data_org_d2],axis=0, ignore_index=False)
#df3 = df3[((df3['label']==0) | (df3['label']==2))]

tweets = df3['tweet'].tolist()
disease = df3['disease'].tolist()
label = df3['label'].tolist()



data = pd.DataFrame({'tweet':tweets, 'disease':disease, 'label':label})
data.info()

# Preprocessing

## Get NRC and VAD Lexicon data 

Default English Stop Words from Different Sources:
https://github.com/igorbrigadir/stopwords

Customize the set of stop words based on Gensim

In [ ]:
# transfer to the DataFrame
VAD_df = pd.read_csv(MY_PATH+'Others/NRC-VAD-Lexicon.txt', sep="\t", header=0)
VAD_df.columns = ['Word', 'Valence', 'Arousal', 'Dominance']
VAD_df.set_index(["Word"], inplace=True)


gensim_sw = [
    'all', 'six', 'just', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'four',  'own', 'through',
    'using', 'fifty', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere',
    'much', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'yourselves', 'under',
    'ours', 'two', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very',
    'de', 'every', 'un',  'front', 'during', 'thus', 'now', 'name', 'regarding',
    'several', 'hereafter', 'did', 'always', 'who', 'didn', 'whither', 'this', 'someone', 'either', 'each', 'become',
    'thereupon', 'sometime', 'side', 'towards', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'doing', 'km',
    'eg', 'some', 'back', 'used', 'up', 'go', 'namely', 'computer', 'are', 'further', 'beyond', 'ourselves', 'yet',
    'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its',
    'everything', 'behind', 'does', 'various', 'above', 'between', 'it', 'neither', 'seemed', 'ever', 'across', 
    'somehow', 'be', 'we', 'full', 'never', 'sixty', 'here', 'otherwise', 'were', 'whereupon', 
    'although', 'found', 'alone', 're', 'along', 'quite', 'fifteen', 'by', 'both', 'about', 'last', 'would',
    'anything', 'via', 'many', 'could', 'thence', 'put', 'against', 'keep', 'etc', 'amount', 'became', 'ltd', 'hence',
    'onto', 'or', 'con', 'among', 'already', 'co', 'afterwards', 'formerly', 'within', 'seems', 'into', 'others',
    'while', 'whatever', 'except', 'down', 'hers', 'everyone', 'done', 'least', 'another', 'whoever', 'moreover',
    'couldnt', 'throughout', 'anyhow', 'yourself', 'three', 'from', 'few', 'together', 'top', 'there', 'due',
    'been', 'next', 'anyone', 'eleven', 'cry', 'call', 'therefore', 'interest', 'then', 'thru', 'themselves',
    'hundred', 'really', 'sincere', 'empty', 'more', 'himself', 'elsewhere', 'mostly', 'on', 'fire', 'am', 'becoming',
    'hereby', 'amongst', 'else', 'part', 'everywhere', 'too', 'kg', 'herself', 'former', 'those',  'me', 'myself',
    'made', 'twenty', 'these', 'was', 'bill', 'cant', 'us', 'until', 'besides', 'nevertheless', 'below', 'anywhere',
    'nine', 'can', 'whether', 'of', 'your', 'toward', 'my', 'say', 'something', 'and', 'whereafter', 'whenever',
    'give', 'almost', 'wherever', 'is', 'describe', 'beforehand', 'herein', 'doesn', 'an', 'as', 'itself', 'at',
    'have', 'in', 'seem', 'whence', 'ie', 'any', 'fill', 'again', 'hasnt', 'inc', 'thereby', 'thin',  'perhaps',
    'latter', 'meanwhile', 'when', 'detail', 'same', 'wherein', 'beside', 'also', 'that', 'other', 'take', 'which',
    'becomes', 'if',  'unless', 'whereas', 'see', 'though', 'may', 'after', 'upon', 'most', 'hereupon',
    'eight', 'serious',  'such', 'why', 'off', 'a', 'don', 'whereby', 'third', 'whole', 'noone', 'make', 'once',
    'sometimes', 'well', 'amoungst', 'yours', 'their', 'rather', 'without', 'so', 'five', 'the', 'first', 'with',
    'not','no','none','nowhere','nobody','nothing','nor', 'however','cannot', 'but', 'you', 'he','they','him','she','i', 'her'
]  

keep = VAD_df.index.tolist()+['not','no','none','nowhere','nobody','nothing','nor','however','cannot','but','you','he','they','him','she','i','her'] 
new_sw = [t for t in gensim_sw if t not in keep]

def remove_sw(s, stopword_set):
    """Remove :const:`~gensim.parsing.preprocessing.STOPWORDS` from `s`.
    Parameters
    ----------
    s : str
    Returns
    -------
    str
    """
    STOPWORDS = frozenset(stopword_set)
    s = utils.to_unicode(s)
    return " ".join(w for w in s.split() if w not in STOPWORDS)

print("The length of Gensim's stopword is", len(gensim_sw))
print("The length of the new set of stopword is", len(new_sw))


### EXPLANATION ###
# This chunk of code list down all the stopwords and filter some out
# from gensim stopwords, they filtered out some words from NRC-VAD-Lexicon.txt


The length of Gensim's stopword is 337
The length of the new set of stopword is 241


## Expand abbreviation and acronyms

In [ ]:
# Expansion, abbreviation and acronyms that often occur in social platform
# e.g. "2b":"to be", "asap":"as soon as possible"
lingo = pd.read_csv(MY_PATH+'Others/lingo_new.csv')
lingo.columns = ['index','abbr','expansion']
lingo_dict = {}
for i in range(len(lingo)):
    abbr = str(lingo['abbr'][i])
    _abbr = "".join(abbr.split())
    if len(re.findall(r"[^0-9A-Za-z']", _abbr))==0:
        if "cancer" not in abbr and len(_abbr)>1:    
            lingo_dict[abbr]=str(lingo['expansion'][i])

# Contraction expansion
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", 
                    "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                    "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                    "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                    "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", 
                    "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "&amp":" and ",
                    }

stem_dict = {"cancers":"cancer", "alzheimers":"alzheimer", "parkinsons":"parkinson", "coughs":"cough", 
             "heartattack":"heart attack", "heartattacks":"heart attack", "depressions":"depression", 
             "fevers":"fever", "headaches":"headache", "strokes":"stroke", "migraines":"migraine",
             "migraineurs":"migraine", "anticancer":"anti cancer", "coughdrops": "cough drop",
             "cancerian":"cancer", "coughed":"cough", "coughing":"cough","hayfever":"hay fever",
             "coughfromfebruarycough":"cough from february cough", "cancercurenow":"cancer cure now",
             "coughpersonaseries":"cough person a series"
             }

# Expansion Function
def expansion(sentence, pp_dict):
    _re = re.compile(r'\b(?:%s)\b' % '|'.join(pp_dict.keys()))
    def replace(match_obj):
        return pp_dict[match_obj.group(0)]
    sentence = _re.sub(replace, sentence)
    return sentence

# Repeated characters were normalized to two repeats to address exaggerations such as ‘looool’ and ‘lool’, and repeated emojis.
def reduce_lengthening(text):
    """
    Replace repeated character sequences of length 2 or greater with sequences
    of length 2.
    """
    pattern = regex.compile(r"(.)\1{1,}")
    return pattern.sub(r"\1\1", text)

### EXPLANATION ###
# Expand the abbreviation and acronyms that often occur in social platform
# Delete character sequence that have length more than 2

In [ ]:
class preprocessing():
    def __init__(self, hashtag_ann=False, stopword_set=None, spell_cor=False):
        self.hashtag_ann = hashtag_ann
        self.stopword = stopword_set
        self.spell_cor = spell_cor
        # terms that will be normalized
        if self.hashtag_ann:
            self.annotation = {"hashtag", 'emphasis', 'censored', "elongated"}
        else:
            self.annotation = {'emphasis', 'censored', "elongated"}
        self.text_processor = TextPreProcessor(
                # terms that will be normalized
                normalize=['url', 'email', 'percent', 'money', 'phone', 'user', 'time', 'date'],
                # terms that will be annotated e.g."allcaps", "repeated", "elongated",
                annotate = self.annotation,
                fix_html=True,  # fix HTML tokens
                # corpus from which the word statistics are going to be used 
                # for word segmentation 
                segmenter="twitter", 
                
                # corpus from which the word statistics are going to be used 
                # for spell correction
                corrector="twitter", 
                
                unpack_hashtags=True,  # perform word segmentation on hashtags
                # unpack_contractions=True,  # Unpack contractions (can't -> can not)   NOT GOOD!
                spell_correct_elong=self.spell_cor,  # spell correction for elongated words
                
                # select a tokenizer. You can use SocialTokenizer, or pass your own
                # the tokenizer, should take as input a string and return a list of tokens
                tokenizer=SocialTokenizer(lowercase=True).tokenize,
                
                # list of dictionaries, for replacing tokens extracted from the text, with other expressions. 
                dicts=[emoticons]
            )
    
    def process(self, tweet):
        # convert to lower case and convert Emojis were converted to string representations
        tweet = emoji.demojize(tweet.lower())
        
        # expanding contractions
        tweet = expansion(tweet, contraction_dict)
        
        # TextPreProcessor defined above
        tweet = " ".join(self.text_processor.pre_process_doc(tweet))
        
        # expanding abbreviations
        tweet = expansion(tweet, lingo_dict)
        # change non-Latin alphabets to Latin alphabets, e.g. "àáãäå" -> "a"
        tweet = unicodedata.normalize('NFKD', tweet).encode('ASCII', 'ignore').decode('utf8').lower()
        # removal of punctuation, 
        tweet = re.sub(r"(?:\W+)", " ", " ".join(tweet.split()))
        # number
        tweet = re.sub(r"(?:\d+)", " ddd ", " ".join(tweet.split()))
        # Repeated words were normalized to two repeats 
        tweet = re.sub(r'\b(\w+)( \1\b)+', r'\1 \1', " ".join(tweet.split()))
        # remove "_"
        tweet = re.sub(r'_', r' ', tweet)
        # replace
        tweet = expansion(tweet, stem_dict)
        if self.stopword:
            # removal of stopwords   
            tweet = remove_sw(tweet, self.stopword)
        return tweet

## Partitioning

In [ ]:
def partition(keyword, s_token):
    '''
    Parameters
    ----------
    keyword: str, the keyword used to partition the sentence, e.g. 'heart attack'
    sent_token: a list of token str, e.g.['i', 'will', 'never', 'forget',....]
    
    Outputs
    ----------
    keyword: a list of token str, e.g. ['heart', 'attack']
    left: a list of token str, left context
    right: a list of token str, right context
    '''
    sent = " ".join(s_token)
    l,k,r = sent.partition(keyword)
    if len(k.split())>0:
        return k.split(), l.split(), r.split()
    else:
        print(k)
        raise ValueError("keyword is not found in the sequence")

## Save tweets after preprocessing

In [ ]:
# -------------- Parameter Setting -------------#
# stopword could be None, gensim_sw, new_sw
prep = preprocessing(hashtag_ann=False, stopword_set=None, spell_cor=False) 


# Initialization of token lists
tweets_token, left_token, right_token = [],[],[]
nokw_index = [] # No keyword. which tweets has no keyword in it
left_max = 0
right_max = 0
# Preprocess each tweet
for i,tweet in enumerate(tweets):
    tweet_clean = prep.process(tweet)
    tweet_token = tweet_clean.split()
    tweet_token = tweet_token[:min(len(tweet_token), MAX_LENGTH)]  # truncate if more than length_limit
    
    if data["disease"][i] not in tweet_clean:
        nokw_index.append(i)
    elif data["disease"][i].split()[0] not in tweet_token:
        nokw_index.append(i)

    else:
        tweets_token.append(tweet_token)
        _, left, right = partition(data["disease"][i], tweet_token)
        left_token.append(left)
        right_token.append(right)
        left_max = max(left_max, len(left))
        right_max = max(right_max, len(right))

# The number of tokens per sentence
token_distribution = [len(t) for t in tweets_token]
# The maximum sequence length
max_length = max(token_distribution)
print("Maximum sequence length =", max_length)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...
Maximum sequence length = 128


In [ ]:
# For tweets after preprcessing but without padding, drop the tweets that 
# does not contain the corresponding keyword
data_tokens = data.drop(nokw_index).reset_index(drop=True)

data_tokens['tokens'] = tweets_token
data_tokens['token_left'] = left_token
data_tokens['token_right'] = right_token

# Save the tweets after preprocessing to Google Drive
data_tokens.to_json(MY_PATH+'TGS data/data_tokens_noSW_D1.json')

## Load the tweets after preprocessing

In [ ]:
json_dataset = MY_PATH + 'TGS data/data_tokens_noSW_D1.json'

df_train = pd.read_json(json_dataset)

if NUM_CLASSES==3:
    pass;
elif NUM_CLASSES==2 and CLASS_NAME[0] == 'Non-health' and CLASS_NAME[1]=='Health':
    # merge label 0 and 1, (binary 01)
    df_train['label'] = df_train.label.apply(lambda x: 0 if x==1 else x)
    df_train['label'] = df_train.label.apply(lambda x: 1 if x==2 else 0)
elif NUM_CLASSES==2 and CLASS_NAME[0] == 'Figurative' and CLASS_NAME[1]=='Health':
    # remove label 1 (binary 02)
    df_train = df_train[df_train['label'] != 1].reset_index(drop=True)
    df_train['label'] = df_train.label.apply(lambda x: 1 if x==2 else 0)
else:
    raise ValueError("Please provide the correct class name: Figurative, Non-health, or Health")

df_train['new_tweet'] = df_train.tokens.apply(lambda x: ' '.join(x))
df_train['disease'] = df_train['disease'].str.lower()
df_train['label'] = df_train['label'].map({'0A': 0, 0: 0, 1:1,2:2,'0':0,'1':1,'2':2})

df_train.head()

In [ ]:
# TGS unlabelled data

data_org_TGS = pd.read_json(MY_PATH+'TGS data/data_tokens_noSW_D1.json')
data_org_TGS['disease'] =  data_org_TGS['disease'].str.lower()
data_org_TGS['new_tweet'] = data_org_TGS.tokens.apply(lambda x: ' '.join(x))

tgs_difference = list(set(df3['disease']) - set(df_train['disease']))
all_disease_nonexists_idx = []
for disease in tgs_difference:
    nonexist_idx = data_org_TGS[ data_org_TGS['disease'] == disease ].index
    all_disease_nonexists_idx.extend(nonexist_idx)

data_org_TGS = data_org_TGS.drop(all_disease_nonexists_idx).reset_index(drop=True)
tgs_diseases = data_org_TGS.disease.values
tgs_texts = data_org_TGS.new_tweet.values

data_org_TGS.head()

,tweet,disease,label,tokens,token_left,token_right,new_tweet
0,"Many of us have witnessed the sad, steady marc...",alzheimer,1,"[many, of, us, have, witnessed, the, sad, stea...","[many, of, us, have, witnessed, the, sad, stea...","[s, disease, as, it, destroys, memory, and, th...",many of us have witnessed the sad steady march...
1,Principal components analysis of agitation out...,alzheimer,1,"[principal, components, analysis, of, agitatio...","[principal, components, analysis, of, agitatio...","[s, disease, url]",principal components analysis of agitation out...
2,I will never go thru my girls phone. Never. No...,heart attack,0,"[i, will, never, go, through, my, girls, phone...","[i, will, never, go, through, my, girls, phone...",[],i will never go through my girls phone never n...
3,Got sumatriptan for this week long migraine. I...,migraine,2,"[got, sumatriptan, for, this, week, long, migr...","[got, sumatriptan, for, this, week, long]","[i, thought, these, were, over, as, a, kid, bu...",got sumatriptan for this week long migraine i ...
4,"The Expert Care Experience: Speech, Swallowing...",parkinson,1,"[the, expert, care, experience, speech, swallo...","[the, expert, care, experience, speech, swallo...","[s, url, via, user]",the expert care experience speech swallowing a...


In [ ]:
tgs_difference = list(set(df3['disease'].str.lower()) - set(df_train['disease'].str.lower()))
tgs_difference

['ptsd', 'ocd']

In [ ]:
train_texts = df_train.new_tweet.values
train_labels = df_train.label.values
train_diseases = df_train.disease.values

In [ ]:
# Unique disease
res_diseases = [''.join(x) for x in list(df_train['disease'])]
unique_disease = list(set(res_diseases))
print(unique_disease)
print(len(unique_disease))

['cough', 'stroke', 'heart attack', 'migraine', 'parkinson', 'fever', 'alzheimer', 'headache', 'cancer', 'depression']
10


# Setup Dataset

In [ ]:
class create_dataset(Dataset):
    def __init__(self, X_text, y, disease_name, tokenizer, max_token_len= 65):
        self.tokenizer = tokenizer
        self.X_text = X_text
        self.y = y
        self.max_token_len = max_token_len
        self.disease = disease_name

    def __len__(self):
        return len(self.X_text)
    
    def __getitem__(self, index: int):
        
        texts = self.X_text[index]
        disease = self.disease[index]

        if self.y is not None:
            labels = self.y[index]
        else: labels = []

        
        
        encoding = self.tokenizer.encode_plus(
          texts,
          add_special_tokens=True, # Add '[CLS]' and '[SEP]'
          max_length=self.max_token_len, # Pad & truncate all sentences.
          return_token_type_ids=True,
          pad_to_max_length = True,
          truncation=True,
          return_attention_mask=True, # Construct attn. masks.
          return_tensors='pt' # Return pytorch tensors.
        )
        ids = encoding['input_ids']
        mask = encoding['attention_mask']
        token_type_ids = encoding["token_type_ids"]
        

        return dict(
          index=index,
          input_ids=torch.tensor(ids.flatten(), dtype=torch.long),
          attention_mask=torch.tensor(mask.flatten(), dtype=torch.long),
          token_type_ids= torch.tensor(token_type_ids.flatten(), dtype=torch.long),
          labels= torch.tensor(labels),
          disease = disease
        )

# Tokenization

In [ ]:

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)


# Pesudo Labelling

In [ ]:
#the variable (output) here is the output of the predictn function
#the shape of the (output) is shown above.
#this equation is from the CATE paper, formula (6)
#on the CATE paper it said that the pseudo label is squaring and normalizing the predictions.
# a while loop is used
def pseudo_label (output):
  #setting the initial index
  i=0
  #creat a list to store the pseudo label result
  pseudo=[]
  while i<len(output):
    y=0
    #the max(output[i]) here is the prediction with higher confidence
    #normization = (x-mean)/std
    x=float(max(output[i]))
    x_mean=float(output[i].mean())
    x_std=float(output[i].std())
    #normalize the prediction and square it (from the cate paper)
    y+=((x-x_mean)/x_std)**2
    #add the result to the list
    pseudo.append(float(y))
    i+=1
    #finally output the list
  return pseudo

#the library here is for the log() function
#this equation is from the CATE paper formula (7)
import math
# for this function, we need two variables, one is the pseudo labels and the other one is the original output of the predictn function
def pseudo_label_loss (pseudo_label,output):
  #set the initial index and the value of the lost as 0
  index=0
  lost=0
  #the big here is the list of the maximum value of prediction
  big=[]
  for i in output:
    #store the max prediction value into the big list
    big.append(float(max(i)))
    #use a while loop to calculate the lost
  while index<len(pseudo_label):
    #the lost = sum of pseudo times the log of pseudo label devided by the absolute value of maximum prediction
    lost+=pseudo_label[index]*(math.log(pseudo_label[index]/abs(big[index])))
    index+=1
    #output the value of lost
  return lost

# Model

In [ ]:

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 

bert_model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, 
    num_labels = NUM_CLASSES, # The number of output labels--2 for binary classification.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClass

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(bert_model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 393 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 1024)
bert.embeddings.position_embeddings.weight               (512, 1024)
bert.embeddings.token_type_embeddings.weight               (2, 1024)
bert.embeddings.LayerNorm.weight                             (1024,)
bert.embeddings.LayerNorm.bias                               (1024,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight        (1024, 1024)
bert.encoder.layer.0.attention.self.query.bias               (1024,)
bert.encoder.layer.0.attention.self.key.weight          (1024, 1024)
bert.encoder.layer.0.attention.self.key.bias                 (1024,)
bert.encoder.layer.0.attention.self.value.weight        (1024, 1024)
bert.encoder.layer.0.attention.self.value.bias               (1024,)
bert.encoder.layer.0.attention.output.dense.weight      (1024, 1024)
bert.encoder.layer.0.attention.output.dense.bias             (

## freeze_layer code

In [ ]:
if freeze_layer_count:
    # We freeze here the embeddings of the model
    for param in bert_model.bert.embeddings.parameters():
        param.requires_grad = False

    if freeze_layer_count != -1:
        # if freeze_layer_count == -1, we only freeze the embedding layer
        # otherwise we freeze the first `freeze_layer_count` encoder layers
        for layer in bert_model.bert.encoder.layer[:freeze_layer_count]:
            for param in layer.parameters():
                param.requires_grad = False

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(bert_model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
# Function to calculate the accuracy of our predictions vs labels

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
def flat_f1_score(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,pred_flat,average='macro')
def flat_precision(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return precision_score(labels_flat,pred_flat,average='macro')
def flat_recall(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return recall_score(labels_flat,pred_flat,average='macro')

def flat_tnfm(preds, labels):
    TNFM = 0
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for i,L in enumerate(labels_flat):
        if L==pred_flat[i]==0:
          TNFM += 1
    unique, counts = np.unique(labels_flat, return_counts=True)
    label_dict = dict(zip(unique, counts))
    if 0 in label_dict.keys():
        count_0 = label_dict[0]
        TNFM_FM = TNFM/count_0
    else:
        TNFM_FM = 0
    
    return TNFM_FM

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:

def evaluation(param, predicted, labels_torch, whether_show = False):

  predicted = np.asarray(predicted)
  labels_torch = np.asarray(labels_torch)
  acc = accuracy_score(predicted, labels_torch)
  f1 = f1_score(labels_torch, predicted, average='macro')
  precision = precision_score(labels_torch, predicted, average='macro')
  recall = recall_score(labels_torch, predicted, average='macro')

  TN = 0
  FP = []
  for i in range(len(labels_torch)):
    if labels_torch[i] == predicted[i] == 0:
      TN += 1
    if labels_torch[i] != 2 and predicted[i] == 2:
      FP.append(labels_torch[i])


  neg_labels = list(labels_torch).count(0)
  if neg_labels == 0:
      TNFM_FM = 0
  else:
      TNFM_FM = TN / list(labels_torch).count(0)
  try:
    FPFM_FP = FP.count(0) / len(FP)
  except:
    #None if no False Postive
    FPFM_FP = None
  
  # classification report

  records = {"micro_f1":acc, "macro_f1":f1, "precision":precision, "recall":recall, "TNFM_FM":TNFM_FM, "FPFM_FP":FPFM_FP}

  if whether_show:
    print('\n'+param + " accuracy (micro f1 score):", records['micro_f1'])
    print(param + " macro f1 score:", records['macro_f1'])
    print(param + " precision:", records['precision'])
    print(param + " recall:", records['recall'])
    print(param + " TNFM_FM:", records['TNFM_FM'])
    print(param + " FPFM_FP:", records['FPFM_FP'])

  return records

def metrics_per_diseases(param, unique_disease, predicted_, diseases_, labels_torch_, whether_show = False):
  # dictionary to record each disease's prediction and actual_label information
  preciction_disease_dict = {}
  actual_disease_dict = {}
  metrics_diseases = {}

  for i in unique_disease:
    preciction_disease_dict[i] = []
    actual_disease_dict[i] = []

  for i in range(len(predicted_)):
    preciction_disease_dict[diseases_[i]].append(int(predicted_[i]))
    actual_disease_dict[diseases_[i]].append(labels_torch_[i])

  for i in unique_disease:
    metrics_diseases[i] = evaluation(param + ' ' + i, preciction_disease_dict[i], actual_disease_dict[i], whether_show)

  return metrics_diseases

In [ ]:
def get_contrastive_loss(input_batch, gamma, target_batch, measure):
  d, nd = 0, 0
  for i, con_feat in enumerate(input_batch):
    for j, con_feat_sub in enumerate(input_batch):
      if i == j:
        continue
      if target_batch[i] == target_batch[j]:
        d += dist(input_batch[i], input_batch[j], measure)
      else:
        diff = gamma - dist(input_batch[i], input_batch[j], measure)
        nd += max(0, diff)
  
  res = d + nd
  return res

In [ ]:
def dist(a, b, dist_measure):
  if dist_measure == 'euclidean':
    eps = 1e-8
    d = 0.5 * ((a - b).var() / (a.var() + b.var() + eps))
    output = d ** 0.5
  elif dist_measure == 'cosine':
    cos = nn.CosineSimilarity(dim=0)
    output = cos(a.to(torch.float64), b.to(torch.float64))
  return output

## Training

In [ ]:

# For each epoch...
def training(train_dataloader, validation_dataloader, model, scheduler, epochs, fold_id, tgs_dataloader = None):
    print("Fold:",fold_id)
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()
    fold_prediction, fold_disease, fold_label = np.empty((0,3), int), [], np.empty((0,3), int)
    all_fold_prediction, all_fold_disease, all_fold_label = np.empty((0,3), int), [], np.empty((0,3), int)
    best_epoch = 0
    best_val_acc = 0
    epoch_nums_ignored = 0
    epoch_threshold = 4
    break_now = False

    for epoch_i in range(0, epochs):
        
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        t0 = time.time()
        total_train_loss_forupdate = 0
        total_train_loss = 0

        total_train_accuracy = 0
        total_train_f1_score = 0
        total_train_precision = 0
        total_train_recall = 0
        total_train_tnfm = 0

        model.train()

        for step, batch in enumerate(train_dataloader):

            elapsed = format_time(time.time() - t0)

            # Report progress.
            sys.stdout.write('\r')
            sys.stdout.write('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            sys.stdout.flush()

            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_token_type_ids = batch['token_type_ids'].to(device)
            b_labels = batch['labels'].to(device)

            model.zero_grad()        


            output = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            
            loss = output.loss

            #add contrastive #
            if enable_CO:
              #loss_co =  get_contrastive_loss(b_input_ids, 1.0, b_labels, 'cosine')
              
              last_hidden_states = output.hidden_states[-1]
              cls_hidden_state = last_hidden_states[:, 0, :]

              #features = o.pooler_output
              labels = b_labels
              loss_co =  get_contrastive_loss(cls_hidden_state, 1.0, labels, 'cosine')

              loss += (Alpha * loss_co)

            #end add contrastive #


            logits = output.logits
            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.'
            if step < len(train_dataloader)-1:
              total_train_loss_forupdate += loss.item()
              total_train_loss += loss.item()
            else:
              total_train_loss += loss.item()
              loss += total_train_loss_forupdate

            #total_train_loss += loss.item()
            #stephen added performance matric - start
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            # accumulate it over all batches.
            total_train_accuracy += flat_accuracy(logits, label_ids)
            total_train_f1_score += flat_f1_score(logits, label_ids)
            total_train_precision += flat_precision(logits, label_ids)
            total_train_recall += flat_recall(logits, label_ids)
            total_train_tnfm += flat_tnfm(logits, label_ids)
            
            # Perform a backward pass to calculate the gradients.
            loss.backward(retain_graph=True)

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()

            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)            
        
        training_time = time.time() - t0

        print("  Average training loss: {0:.5f}, elapsed: {0:.2f}s".format(avg_train_loss, training_time))
        
        avg_train_accuracy = total_train_accuracy / len(train_dataloader)
        print("  Accuracy: {0:.5f}".format(avg_train_accuracy))
        avg_train_f1_score = total_train_f1_score / len(train_dataloader)
        print('F1 score :',avg_train_f1_score)
        avg_train_precision = total_train_precision / len(train_dataloader)
        print('Precision :',avg_train_precision)
        avg_train_recall = total_train_recall / len(train_dataloader)
        print('recall :',avg_train_recall)
        avg_train_tnfm = total_train_tnfm / len(train_dataloader)
        print('TNFM :',avg_train_tnfm)

        if (tgs_dataloader):
            print("")
            print("Running TGS...")
            beta_loss_st  = beta_loss
            model.eval()
            t0 = time.time()
            tgs_predictions = []
            for batch in tgs_dataloader:
                b_input_ids = batch['input_ids'].to(device)
                b_input_mask = batch['attention_mask'].to(device)
                b_token_type_ids = batch['token_type_ids'].to(device)
                with torch.no_grad():
                    tgs_output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                    tgs_logits = tgs_output.logits
                    tgs_logits = tgs_logits.detach().cpu()
                    tgs_logits_softmax = torch.softmax(tgs_logits, dim=-1).numpy()
                    tgs_predictions.append(tgs_logits_softmax)
            tgs_flat_predictions = [item for sublist in tgs_predictions for item in sublist]
            soft_pseudo_labels = pseudo_label(tgs_flat_predictions)
            loss_st = pseudo_label_loss(soft_pseudo_labels,tgs_flat_predictions)
            print('loss_st:', loss_st)
            total_loss = loss + (loss_st * beta_loss_st)

            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        


        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # ==============used to capture the best validation score, initialised at -inf
        

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_f1_score = 0
        total_eval_precision = 0
        total_eval_recall = 0
        total_eval_tnfm = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
                        
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_token_type_ids = batch['token_type_ids'].to(device)
            b_labels = batch['labels'].to(device)
            b_diseases = batch['disease']
            
            with torch.no_grad():        
                
                output = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
                loss = output.loss
                logits = output.logits
                
                
            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            
            
            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)

            total_eval_f1_score += flat_f1_score(logits, label_ids)
            total_eval_precision += flat_precision(logits, label_ids)
            total_eval_recall += flat_recall(logits, label_ids)
            total_eval_tnfm += flat_tnfm(logits, label_ids)

            #=====below part is added for generate disease-wise result=======#
            fold_disease += b_diseases
            fold_label = np.append(fold_label, label_ids) 

            flat_prediction = np.argmax(logits, axis=1).flatten()
            fold_prediction = np.append(fold_prediction, flat_prediction)
            #=====above part is added for generate disease-wise result=======#


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("Accuracy: {0:.5f}".format(avg_val_accuracy))
        avg_val_f1_score = total_eval_f1_score / len(validation_dataloader)
        print('F1 score :',avg_val_f1_score)
        avg_val_precision = total_eval_precision / len(validation_dataloader)
        print('Precision :',avg_val_precision)
        avg_val_recall = total_eval_recall / len(validation_dataloader)
        print('recall :',avg_val_recall)
        avg_val_tnfm = total_eval_tnfm / len(validation_dataloader)
        print('TNFM :',avg_val_tnfm)

          
        #=======select best epoch=========#
        if avg_val_f1_score > best_val_acc:
            epoch_nums_ignored = 0
            best_val_acc = avg_val_f1_score
            all_fold_disease = fold_disease
            all_fold_label = fold_label
            all_fold_prediction = fold_prediction
            
            print(len(all_fold_prediction))
            best_model = {
                'epoch': epoch_i + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'validation_pred':all_fold_prediction,
                'y_test':all_fold_label,
                'RNG':torch.get_rng_state()
            }
            #torch.save(best_model, MY_PATH+'/Files_outputs/'+MODEL_ID+ '_D1_binary_'+ str(fold_id) +'.pt')
            best_epoch = epoch_i + 1
            best_trained_model = model
        
        elif avg_val_f1_score <= best_val_acc:
          epoch_nums_ignored += 1
          if epoch_nums_ignored >= epoch_threshold:
            break_now = True


        fold_disease = []
        fold_label = np.empty((0,3), int)
        fold_prediction = np.empty((0,3), int)      
        #=======end select best epoch=========#    

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        
        # Measure how long the validation run took.
        validation_time = time.time() - t0
        
        print("  Validation Loss: {0:.5f}, elapsed: {0:.2f}s".format(avg_val_loss, validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'fold':fold_id,
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Training Accuracy': avg_train_accuracy,
                'Training F1 score': avg_train_f1_score,
                'Training Precision': avg_train_precision,
                'Training Recall': avg_train_recall,
                'Training TNFM': avg_train_tnfm,
                'Valid. Loss': avg_val_loss,
                'Valid. Accuracy': avg_val_accuracy,
                'Valid. F1 score': avg_val_f1_score,
                'Valid. Precision': avg_val_precision,
                'Valid. Recall': avg_val_recall,
                'Valid. TNFM': avg_val_tnfm,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )
        result_history = pd.DataFrame(training_stats)
        result_history.to_csv(MY_PATH+ 'Point8/' +MODEL_ID+'-'+str(fold_id)+'.csv', index=False)

        if break_now:
          print("Exiting out of epoch {}".format(epoch_i+1))
          break

    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    return model, training_stats, all_fold_disease, all_fold_label, all_fold_prediction, best_epoch

In [ ]:
import gc
gc.collect()
    
torch.cuda.empty_cache()


In [ ]:
# Training

splits = list(KFold(n_splits=n_splits, shuffle=True, random_state=SEED).split(train_texts, train_labels))

torch.cuda.empty_cache()
val_pred_all = []
all_fold_prediction, all_fold_disease, all_fold_label = np.empty((0,3), int), [], np.empty((0,3), int)
#this stores the best epoch results for each fold, for all the folds
best_epoch_results_all_folds = []

for fold_id, (train_idx, valid_idx) in enumerate(splits):
    x_train_fold = np.array(train_texts)[train_idx.astype(int)]
    y_train_fold = np.array(train_labels)[train_idx.astype(int)]
    disease_train = train_diseases[train_idx.astype(int)]
    x_val_fold = np.array(train_texts)[valid_idx.astype(int)]
    y_val_fold = np.array(train_labels)[valid_idx.astype(int)]
    disease_val = train_diseases[valid_idx.astype(int)]

    model = copy.deepcopy(bert_model)
    model.to(device)
    
    
    train_dataset = create_dataset(x_train_fold, y_train_fold, disease_train,tokenizer, max_token_len=MAX_LENGTH)
    val_dataset = create_dataset(x_val_fold, y_val_fold, disease_val, tokenizer, max_token_len=MAX_LENGTH)

    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = BATCH_SIZE # Trains with this batch size.
            )
    
    if enable_TGS:
        #set tgs len to 65 otherwise is too much
        tgs_dataset = create_dataset(tgs_texts, None, tgs_diseases, tokenizer, max_token_len=65)
        tgs_dataloader = DataLoader(
            tgs_dataset,
            sampler = SequentialSampler(tgs_dataset),
            batch_size = BATCH_SIZE
            )
    else:
        tgs_dataloader = None


    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataloader = DataLoader(
                val_dataset, # The validation samples.
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                batch_size = BATCH_SIZE # Evaluate with this batch size.
            )
    total_steps = len(train_dataloader) * EPOCH
    
    optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    trained_model, result_per_epoch, fold_diseases, fold_label, fold_prediction, best_epoch = training(train_dataloader, validation_dataloader, model, scheduler, EPOCH, fold_id, tgs_dataloader)
    #=====below part is added for generate disease-wise result=======#
    all_fold_disease += fold_diseases
    #print(fold_label)
    all_fold_label = np.append(all_fold_label,fold_label)
    #flat_predictions = [item for sublist in fold_prediction for item in sublist]
    #flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    all_fold_prediction=np.append(all_fold_prediction,fold_prediction) 
    #=====above part is added for generate disease-wise result=======#
    
    
    best_epoch_results_all_folds.append(result_per_epoch[best_epoch - 1])


Fold: 0
======== Epoch 1 / 10 ========
Training...
  Batch   819  of    820.    Elapsed: 0:05:50.  Average training loss: 6.01136, elapsed: 6.01s
  Accuracy: 0.83049
F1 score : 0.8080401870036015
Precision : 0.831936701509872
recall : 0.827694541231125
TNFM : 0.8240606852497095

Running TGS...
loss_st: 11634.890517394973

Running Validation...
Accuracy: 0.94695
F1 score : 0.9350555406652964
Precision : 0.9425464576074334
recall : 0.9420383275261328
TNFM : 0.960569105691057
1639
  Validation Loss: 0.18703, elapsed: 0.19s
======== Epoch 2 / 10 ========
Training...
  Batch   819  of    820.    Elapsed: 0:05:47.  Average training loss: 5.75689, elapsed: 5.76s
  Accuracy: 0.96143
F1 score : 0.9528584762121356
Precision : 0.9588734030197436
recall : 0.9588951800232282
TNFM : 0.957168989547038

Running TGS...
loss_st: 11465.416071084248

Running Validation...
Accuracy: 0.94451
F1 score : 0.9329062807111582
Precision : 0.9386062717770036
recall : 0.9418902439024391
TNFM : 0.9609291521486645
  

In [ ]:
#save results of best epoch of each fold only

fold_no = [np.amax(np.array(best_epoch_results_all_folds[x]['fold'])) for x in range(len(best_epoch_results_all_folds))]
best_epoch_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['epoch'])) for x in range(len(best_epoch_results_all_folds))]
best_train_acc_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Training Accuracy'])) for x in range(len(best_epoch_results_all_folds))]
best_val_acc_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Valid. Accuracy'])) for x in range(len(best_epoch_results_all_folds))]
best_train_f1_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Training F1 score'])) for x in range(len(best_epoch_results_all_folds))]
best_val_f1_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Valid. F1 score'])) for x in range(len(best_epoch_results_all_folds))]
best_train_prec_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Training Precision'])) for x in range(len(best_epoch_results_all_folds))]
best_val_prec_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Valid. Precision'])) for x in range(len(best_epoch_results_all_folds))]
best_train_recall_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Training Recall'])) for x in range(len(best_epoch_results_all_folds))]
best_val_recall_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Valid. Recall'])) for x in range(len(best_epoch_results_all_folds))]
best_train_tnfm_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Training TNFM'])) for x in range(len(best_epoch_results_all_folds))]
best_val_tnfm_5_fold = [np.amax(np.array(best_epoch_results_all_folds[x]['Valid. TNFM'])) for x in range(len(best_epoch_results_all_folds))]

best_fold_results = pd.DataFrame({'Fold':[x + 1 for x in range(len(fold_no))],
                             'Epoch' : best_epoch_5_fold,
                             'Train_Acc': best_train_acc_5_fold,
                             'Validation_Acc': best_val_acc_5_fold,     
                             'Train_f1': best_train_f1_5_fold,
                             'Validation_f1': best_val_f1_5_fold,
                             'Train_Prec': best_train_prec_5_fold,
                             'Validation_Prec': best_val_prec_5_fold,
                             'Train_Recall': best_train_recall_5_fold,
                             'Validation_Recall': best_val_recall_5_fold,
                             'Train_TNFM': best_train_tnfm_5_fold,
                             'Validation_TNFM': best_val_tnfm_5_fold})

best_fold_results.to_csv(MY_PATH+ 'Point8/' + MODEL_ID+ '_'+str(n_splits)+'_Fold_Results.csv', index=False)


In [ ]:
print(len(unique_disease))
print(len(all_fold_prediction))
print(len(all_fold_disease))
print(len(all_fold_label))

10
8193
8193
8193


# Disease-wise Result

In [ ]:
from sklearn.metrics import classification_report
dis_wise_res = metrics_per_diseases("All", unique_disease, all_fold_prediction, all_fold_disease, all_fold_label, whether_show = False)

In [ ]:
keywords = ['macro_f1', 'precision', 'recall', 'TNFM_FM']

diseasewiseDict = {}

for ud in unique_disease:
    temp = {}
    for x in keywords:
        temp[x] = dis_wise_res[ud][x]
    diseasewiseDict[ud] = temp
        

print(diseasewiseDict)
diseasewisedf = pd.DataFrame(diseasewiseDict)
diseasewisedf.to_csv(MY_PATH+ 'Point8/' + MODEL_ID+'_Fixed_Diseasewise' + '.csv')

{'cough': {'macro_f1': 0.9849089127573526, 'precision': 0.9853215513009328, 'recall': 0.9844992718718495, 'TNFM_FM': 0.9911504424778761}, 'stroke': {'macro_f1': 0.9540457494025265, 'precision': 0.9536890645586298, 'recall': 0.954407495381367, 'TNFM_FM': 0.9643705463182898}, 'heart attack': {'macro_f1': 0.9324389377553023, 'precision': 0.9431735465951536, 'recall': 0.9225113225464773, 'TNFM_FM': 0.9837786259541985}, 'migraine': {'macro_f1': 0.9465497737556561, 'precision': 0.9568138836355545, 'recall': 0.9370915206608051, 'TNFM_FM': 0.8878504672897196}, 'parkinson': {'macro_f1': 0.9164333706606943, 'precision': 0.9227763569195795, 'recall': 0.9105270419426048, 'TNFM_FM': 0.8541666666666666}, 'fever': {'macro_f1': 0.9640637843169488, 'precision': 0.9634825217215599, 'recall': 0.9647785297455017, 'TNFM_FM': 0.9597989949748744}, 'alzheimer': {'macro_f1': 0.9211368841982714, 'precision': 0.9389986824769434, 'recall': 0.9064207650273224, 'TNFM_FM': 0.8333333333333334}, 'headache': {'macro_f1